In [2]:
url='https://mtc1.uk/Entry/CSRTC19/EntryList.php'

In [15]:
import unicodedata

def cleanString(s):
    s = unicodedata.normalize("NFKD", str(s))
    #replace multiple whitespace with single space
    s = ' '.join(s.split())
    
    return s
    

In [19]:
import requests
import pandas as pd
import lxml.html as LH
from bs4 import BeautifulSoup

html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml') # Parse the HTML as a string
    
tables = soup.find_all('table')
#tables = LH.fromstring(html).xpath('//table')
entries = pd.read_html('<html><body>{}</body></html>'.format(tables[0]))[0]
#entries['badge'] = [img.find('img')['src'] for img in tables[0].findAll("td", {"class": "final-results-icon"}) ]
#entries.dropna(how='all', axis=1, inplace=True)
entries.head()

,No,Class,DriverNationality ~ TownClubsChampionships,Co-driver Nationality ~ TownClubsChampionships,Car CC ~ Colour ~ Registration ~ Year Sponsor
0,1.0,E,Thomas Preston Lancashire PDMC / CMC,Carl Williamson Bradford BTRDA / CMC,Skoda Fabia R51600T ~ Black/Purple ~ SP0541 ~ ...
1,2.0,E,Robert Swann Bishops StortfordWickford AC,Steve McNulty IlfordChelmsford Motor Club,Mitsubishi Mirage1598T ~ Yellow ~ GM65 OHJ ~ 2...
2,4.0,E,Michael O'Brien St AlbansLondon Irish,Mark Glennerster SwindonLondon Irish,Ford Focus WRC 20081998T ~ Red ~ EA07 RGZ ~ 20...
3,5.0,D,Oliver Davies BridgnorthOwen MC / CMC,Ben Innes KidderminsterLudlow Castle MC / CMC,Ford Escort2500 ~ Bullet Blue ~ FCJ 249W ~ tbc...
4,8.0,E,James Self North PethertonBolton,Ian Davis Llanvetherine BTRDA / HOVAC / CMC,Ford Fiesta R51598T ~ White ~ PX15HYY ~ 2015El...


In [20]:
classes = [div.text for div in tables[0].findAll("div", {"class": "entry-class"}) ]
print(classes)

['E', 'E', 'E', 'D', 'E', 'E', 'D', 'E', 'D', 'E', 'HK', 'D', 'D', 'E', 'E', 'E', 'C', 'C', 'C', 'E', 'E', 'E', 'C', 'D', 'E', 'E', 'C', 'E', 'A', 'A', 'C', 'H3', 'H2', 'H2', 'H4b', 'C', 'C', 'E', 'HK', 'E', 'C', 'B', 'B', 'C', 'C', 'D', 'D', 'C', 'B', 'B', 'B', 'E', 'D', 'HK', 'E', 'E', 'A', 'C', 'A', 'C', 'C', 'E', 'C', 'C', 'B', 'C', 'B', 'E', 'C', 'C', 'H3', 'E', 'HK', 'E', 'H3', 'A', 'H3', 'B', 'B', 'B', 'A', 'A', 'A', 'H4b', 'B', 'B', 'B', 'A', 'C', 'C', 'C', 'B', 'H1', 'B', 'B', 'H1', 'C', 'E', 'C', 'H3', 'HK', 'A', 'A', 'C', 'B', 'A', 'B', 'D', 'A', 'C', 'C', 'B', 'D', 'A', 'C', 'E', 'H1', 'C', 'E', 'H2', 'H2', 'H2', 'H3', 'E', 'HK']


In [29]:
row = tables[0].findAll("tr")[1]
cells= row.findAll('td')

entry = cells[1].text

divs = cells[2].findAll('div')
driver = cleanString( divs[0].text )
driver_nationality = divs[1].find('img')['alt']
driver_location = divs[1].text.strip()
driver_club = cleanString( divs[2].text )
#entry, driver, driver_nationality, driver_location, driver_club

divs = cells[3].findAll('div')
navigator = cleanString( divs[0].text )
navigator_nationality = divs[1].find('img')['alt']
navigator_location = divs[1].text.strip()
navigator_club = cleanString( divs[2].text )
#navigator, navigator_nationality, navigator_location, navigator_club

divs = cells[4].findAll('div')
entry_name = cleanString( divs[0].text )
entry_size = 
entry_colur
entry_type = cleanString( divs[1].text )
entry_sponsor = cleanString( divs[2].text )
entry_name, entry_type, entry_sponsor

('Skoda Fabia R5', '1600T ~ Black/Purple ~ SP0541 ~ 2018', 'Hippo Motor Group')

In [70]:
df = pd.DataFrame()
for row in tables[0].findAll("tr"):
    cells= row.findAll('td')
    if not cells:
        continue

    entry = cells[1].text

    divs = cells[2].findAll('div')
    driver = cleanString( divs[0].text )
    driver_nationality = divs[1].find('img')['alt'] if divs[1].find('img') else ''
    driver_location = divs[1].text.strip()
    driver_club = cleanString( divs[2].text )
    #entry, driver, driver_nationality, driver_location, driver_club

    divs = cells[3].findAll('div')
    navigator =  divs[0].text
    navigator_nationality = divs[1].find('img')['alt'] if divs[1].find('img') else ''
    navigator_location = divs[1].text.strip()
    navigator_club = divs[2].text 
    #navigator, navigator_nationality, navigator_location, navigator_club

    divs = cells[4].findAll('div')
    entry_name =  divs[0].text
    entry_type = divs[1].text
    entry_sponsor = divs[2].text
    entry_name, entry_type, entry_sponsor
    
    entry_bits = entry_type.split('~')
    entry_cc = entry_bits[0]
    entry_colour = entry_bits[1]
    entry_reg = entry_bits[2]
    entry_year = entry_bits[3]
    
    df = pd.concat([df, pd.DataFrame([[entry, driver, driver_nationality, driver_location, driver_club,navigator,
                    navigator_nationality, navigator_location, navigator_club,
                   entry_name, entry_type, entry_sponsor, entry_cc, entry_colour, entry_reg,entry_year ]] )])
    
df.columns = ['Class', 'Driver','Driver_Nationality','Driver_Town','Driver_Clubs',
              'Co-driver','Co-driver_Nationality','Co-driver_Town','Co-driver_Clubs',
              'Car','Details', 'Sponsor','CC','Colour','Registration','Year']
df=df[1:]
df.head()

,Class,Driver,Driver_Nationality,Driver_Town,Driver_Clubs,Co-driver,Co-driver_Nationality,Co-driver_Town,Co-driver_Clubs,Car,Details,Sponsor,CC,Colour,Registration,Year
0,E,Thomas Preston,British,Lancashire,PDMC / CMC,Carl Williamson,British,Bradford,BTRDA / CMC,Skoda Fabia R5,1600T ~ Black/Purple ~ SP0541 ~ 2018,Hippo Motor Group,1600T,Black/Purple,SP0541,2018
0,E,Robert Swann,English,Bishops Stortford,Wickford AC,Steve McNulty,British,Ilford,Chelmsford Motor Club,Mitsubishi Mirage,1598T ~ Yellow ~ GM65 OHJ ~ 2015,"Elegant Hotels, Blue Sky luxury, R.A.SWANN LTD",1598T,Yellow,GM65 OHJ,2015
0,E,Michael O'Brien,Irish,St Albans,London Irish,Mark Glennerster,Welsh,Swindon,London Irish,Ford Focus WRC 2008,1998T ~ Red ~ EA07 RGZ ~ 2007,M O'Brien group of companies / Willow Hire,1998T,Red,EA07 RGZ,2007
0,D,Oliver Davies,British,Bridgnorth,Owen MC / CMC,Ben Innes,British,Kidderminster,Ludlow Castle MC / CMC,Ford Escort,2500 ~ Bullet Blue ~ FCJ 249W ~ tbc,"Howells Automotive, Hewitts Salvage",2500,Bullet Blue,FCJ 249W,tbc
0,E,James Self,British,North Petherton,Bolton,Ian Davis,British,Llanvetherine,BTRDA / HOVAC / CMC,Ford Fiesta R5,1598T ~ White ~ PX15HYY ~ 2015,Elite Cameron,1598T,White,PX15HYY,2015


In [71]:
df.to_csv('corbeau19_entries.csv',index=False)

In [72]:
!ls *.csv

corbeau19_entries_archive.csv
